**[1] Get lowest local price**



*   Google Places API key




In [4]:
import requests
from bs4 import BeautifulSoup

In [58]:
GOOGLE_API_KEY = "Your_GOOGLE_PLACES_API_key"



*   To check latitude and longitude of Villages - The Soul Of India




In [59]:
def get_lat_long(place_name):
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={place_name}&key={GOOGLE_API_KEY}"
    response = requests.get(url).json()

    if "results" in response and response["results"]:
        location = response["results"][0]["geometry"]["location"]
        latitude = location["lat"]
        longitude = location["lng"]
        return latitude, longitude
    return None, None

place_name = "Village - The Soul of India, Hicksville, NY"
latitude, longitude = get_lat_long(place_name)
print(f"Latitude: {latitude}, Longitude: {longitude}")


Latitude: 40.7666595, Longitude: -73.5236632


* Get name, address, open times, menu items, & prices for Village

In [60]:
location = "40.7666595,-73.5236632"

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&keyword=Village - The Soul of India&key={GOOGLE_API_KEY}"
response = requests.get(url).json()
print(response)

def get_village_details():
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if data.get("results"):
            village = data["results"][0]
            name = village.get("name")
            address = village.get("vicinity")
            opening_hours = village.get("opening_hours", {}).get("open_now", "Not available")

            print(f"Name: {name}")
            print(f"Address: {address}")
            print(f"Open Now: {'Yes' if opening_hours else 'No'}")

        else:
            print("No details found for the Village restaurant.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

get_village_details()


{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 40.7666595, 'lng': -73.5236632}, 'viewport': {'northeast': {'lat': 40.76800487989271, 'lng': -73.52230742010727}, 'southwest': {'lat': 40.76530522010727, 'lng': -73.52500707989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet', 'name': 'Village - The Soul of India', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115853018819933675219">niti</a>'], 'photo_reference': 'AWYs27x7rRSC-IgJGZ5vWM07soK4tEDQd7kwC74tLvZMhsF0jGR9XYsGgtVLbSSVnuw8Sok4wd8c5AqmFh0olbhY3O5vD9cdD0jCsFSVA7vvYg7KySqHxEFLFtStaxeBEowBMln7wfGELBNOEHd0oR50hgghryZtJbr5AMO1mbc8NAVqU7oH', 'width': 4032}], 'place_id': 'ChIJPYSDLXWBwokRHLcHIl02Kh8', 'plus_code': {'co

In [61]:
place_id = 'ChIJPYSDLXWBwokRHLcHIl02Kh8'

place_details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,opening_hours&key={GOOGLE_API_KEY}"

def get_detailed_opening_hours():
    try:
        response = requests.get(place_details_url)
        response.raise_for_status()
        data = response.json()
        print(data)

        if data.get('result', {}).get('opening_hours'):
            opening_hours = data['result']['opening_hours'].get('weekday_text', 'Not available')
            print("Opening Hours:")
            if isinstance(opening_hours, list):
                for day in opening_hours:
                    print(f"   {day}")
            else:
                print("No detailed hours available.")
        else:
            print("No detailed opening hours found for this place.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

get_detailed_opening_hours()

{'html_attributions': [], 'result': {'name': 'Village - The Soul of India', 'opening_hours': {'open_now': False, 'periods': [{'close': {'day': 0, 'time': '1500'}, 'open': {'day': 0, 'time': '1130'}}, {'close': {'day': 0, 'time': '2150'}, 'open': {'day': 0, 'time': '1700'}}, {'close': {'day': 1, 'time': '1500'}, 'open': {'day': 1, 'time': '1130'}}, {'close': {'day': 1, 'time': '2200'}, 'open': {'day': 1, 'time': '1700'}}, {'close': {'day': 3, 'time': '1500'}, 'open': {'day': 3, 'time': '1130'}}, {'close': {'day': 3, 'time': '2200'}, 'open': {'day': 3, 'time': '1700'}}, {'close': {'day': 4, 'time': '1500'}, 'open': {'day': 4, 'time': '1130'}}, {'close': {'day': 4, 'time': '2200'}, 'open': {'day': 4, 'time': '1700'}}, {'close': {'day': 5, 'time': '1500'}, 'open': {'day': 5, 'time': '1130'}}, {'close': {'day': 5, 'time': '2200'}, 'open': {'day': 5, 'time': '1700'}}, {'close': {'day': 6, 'time': '1500'}, 'open': {'day': 6, 'time': '1130'}}, {'close': {'day': 6, 'time': '2150'}, 'open': {'da

In [64]:
menu_url = "https://villagesoulofindia.com/app/order/menu"

def fetch_menu(menu_url):
    try:
        response = requests.get(menu_url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, "html.parser")
        menu_section = soup.find_all("div", class_="col-lg-3 col-md-4 col-sm-10 offset-md-0 offset-sm-1")

        menu_items = []
        for item in menu_section:
            name = item.find("h5").get_text(strip=True)
            description = item.find("span").get_text(strip=True)
            price = item.find("div", class_="text-muted").get_text(strip=True)
            menu_items.append({"name": name, "description": description, "price": price})

        return menu_items

    except requests.exceptions.RequestException as e:
        return f"Error fetching the menu: {e}"

menu = fetch_menu(menu_url)
if isinstance(menu, list):
    print("Menu Items and Prices:")
    for item in menu:
        print(f"Name: {item['name']}\n Price: {item['price']}")
else:
    print(menu)


Menu Items and Prices:
Name: Rasam
 Price: $ 4.95
Name: Sambar
 Price: $ 4.95
Name: Mulligatawny Soup
 Price: $ 4.95
Name: Garden Soup
 Price: $ 4.95
Name: Kachumbar Salad
 Price: $ 4.95
Name: Garden Salad
 Price: $ 4.95
Name: Veg Manchow soup
 Price: $ 4.95
Name: Tomato Soup
 Price: $ 4.95
Name: Katchumber Salad
 Price: $ 4.95
Name: Idly
 Price: $ 5.95
Name: Masala Idly
 Price: $ 7.95
Name: Mini Idly
 Price: $ 5.95
Name: Ghee fried Idly
 Price: $ 7.95
Name: Podi Idly
 Price: $ 6.95
Name: Chilli Idly
 Price: $ 7.95
Name: Idly in sambar bowl
 Price: $ 6.95
Name: Idly in Rasam bowl
 Price: $ 6.95
Name: Dami Idly
 Price: $ 5.95
Name: Dahi Idly
 Price: $ 5.95
Name: Medu Vada
 Price: $ 5.95
Name: Medu Vada in sambar bowl
 Price: $ 5.99
Name: Medu Vada in Rasam Bowl
 Price: $ 5.99
Name: Combo
 Price: $ 5.99
Name: Combo in Sambar Bowl
 Price: $ 5.99
Name: Masala Vada
 Price: $ 5.95
Name: Dahi Vada
 Price: $ 4.95
Name: Batata Vada
 Price: $ 4.95
Name: Mysore Bonda
 Price: $ 4.95
Name: Chennai 

* Get top-rated 5 restaurants in 2 km with similar menu items

In [45]:
def get_top_rated_restaurants():
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if data.get("results"):
            sorted_restaurants = sorted(data["results"], key=lambda x: x.get("rating", 0), reverse=True)
            top_restaurants = sorted_restaurants[:5]

            print("Top-rated restaurants in the vicinity:")
            for i, restaurant in enumerate(top_restaurants, start=1):
                name = restaurant.get("name")
                address = restaurant.get("vicinity")
                rating = restaurant.get("rating", "N/A")
                user_ratings_total = restaurant.get("user_ratings_total", "N/A")
                types = ', '.join(restaurant.get("types", []))
                price_level = restaurant.get("price_level", "N/A")

                print(f"{i}. Name: {name}")
                print(f"   Address: {address}")
                print(f"   Types: {types}")
                print("-" * 40)

        else:
            print("No nearby restaurants found.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

get_top_rated_restaurants()

Top-rated restaurants in the vicinity:
1. Name: Sakana
   Address: 68 North Broadway, Hicksville
   Types: bar, restaurant, food, point_of_interest, establishment
----------------------------------------
2. Name: Smashburger
   Address: 180 West Old Country Road, Hicksville
   Types: restaurant, food, point_of_interest, store, establishment
----------------------------------------
3. Name: Chuck E. Cheese
   Address: 11-15 Hanover Place Delco Plaza, Hicksville
   Types: restaurant, food, point_of_interest, establishment
----------------------------------------
4. Name: Panera Bread
   Address: 201 Broadway Mall, Hicksville
   Types: cafe, bakery, meal_takeaway, restaurant, food, point_of_interest, store, establishment
----------------------------------------
5. Name: Sunshine Hicksville
   Address: 275 West Old Country Road, Hicksville
   Types: restaurant, food, point_of_interest, establishment
----------------------------------------


* Comapring the price of Village with other 5 restaurants

In [48]:
def get_top_rated_restaurants_and_compare_price():
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if data.get("results"):
            sorted_restaurants = sorted(data["results"], key=lambda x: x.get("rating", 0), reverse=True)

            top_restaurants = sorted_restaurants[:5]
            print("Top-rated restaurants in the vicinity:")
            for i, restaurant in enumerate(top_restaurants, start=1):
                name = restaurant.get("name")
                address = restaurant.get("vicinity")
                rating = restaurant.get("rating", "N/A")
                user_ratings_total = restaurant.get("user_ratings_total", "N/A")
                types = ', '.join(restaurant.get("types", []))  # Restaurant type, e.g., "Indian, Vegetarian"
                price_level = restaurant.get("price_level", "N/A")  # Price level (1-4)

                print(f"{i}. Name: {name}")
                print(f"   Rating: {rating} ({user_ratings_total} reviews)")
                print(f"   Price Level: {price_level}")

                compare_price_level(price_level, name)
        else:
            print("No nearby restaurants found.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def compare_price_level(price_level, restaurant_name):
    village_price_level = 2  # Price level of Village - The Soul of India
    if price_level != "N/A":
        if price_level == village_price_level:
            comparison = "same"
        elif price_level > village_price_level:
            comparison = "higher"
        else:
            comparison = "lower"
        print(f"   Price level comparison with Village - The Soul of India: {comparison}")
    else:
        print(f"   Price level data not available for {restaurant_name}")
    print("-" * 40)

get_top_rated_restaurants_and_compare_price()

Top-rated restaurants in the vicinity:
1. Name: Sakana
   Rating: 4.5 (383 reviews)
   Price Level: 1
   Price level comparison with Village - The Soul of India: lower
----------------------------------------
2. Name: Smashburger
   Rating: 4.3 (1213 reviews)
   Price Level: 2
   Price level comparison with Village - The Soul of India: same
----------------------------------------
3. Name: Chuck E. Cheese
   Rating: 4.2 (1123 reviews)
   Price Level: 2
   Price level comparison with Village - The Soul of India: same
----------------------------------------
4. Name: Panera Bread
   Rating: 4.2 (750 reviews)
   Price Level: 2
   Price level comparison with Village - The Soul of India: same
----------------------------------------
5. Name: Sunshine Hicksville
   Rating: 4.2 (574 reviews)
   Price Level: 2
   Price level comparison with Village - The Soul of India: same
----------------------------------------


In the vicinity, **Sakana** is the most affordable with a price level of 1, offering a lower cost compared to **Village - The Soul of India**. 

The rest—Smashburger, Chuck E. Cheese, Panera Bread, and Sunshine Hicksville—are similarly priced to Village, offering moderate pricing with good ratings.